___
___
___
# Question SQL
___
___
___

## INIT
### imports
### connecteur + curseur

In [72]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'azertyuiop',
    database = 'my_imdb'
)

cur = conn.cursor()


___
### 1) Quel est le film le plus long ?

In [73]:
request = """
SELECT CONCAT("Le film le plus long de la BDD est “",
              title,
              "” avec ses ",
              TRUNCATE(duration / 60, 0),
              "h ",
              duration - TRUNCATE(duration / 60, 0) * 60,
              "min")
  FROM movies250 m
 ORDER BY duration DESC
 LIMIT 1;
"""

cur.execute(request)
results = cur.fetchall()

for row in results:
    print(row[0])

Le film le plus long de la BDD est “Shoah” avec ses 9h 26min


___
### 2) Quels sont les 5 films les mieux notés ?

In [74]:
x = 5 # nombre de films

request = f"""
SELECT RPAD(CONCAT("“",
              title,
              "”"), (SELECT MAX(title_length)
                       FROM (SELECT LENGTH(title) title_length
                               FROM movies250
                              ORDER BY score DESC
                              LIMIT {x}) sr) + 2, "-"),
       CONCAT(score, "/10")
  FROM movies250 m
 ORDER BY score DESC
 LIMIT {x};
"""

cur.execute(request)
results = cur.fetchall()

print(f"Les {x} films les mieux notés sont (dans l'ordre) :")
rang = 0
for row in results:
    rang += 1
    print(rang, ":", row[0], row[1])

Les 5 films les mieux notés sont (dans l'ordre) :
1 : “The Counterfeit Kid” 9.7/10
2 : “The Home Game”------ 9.5/10
3 : “My Dead Friend Zoe”- 9.5/10
4 : “Desert Road”-------- 9.4/10
5 : “Wedding 93”--------- 9.4/10


___
### 3) Dans combien de films a joué Morgan Freeman ? Tom Cruise ?

In [75]:
request = f"""
SELECT IFNULL(ma.actor_name, "TOTAL") Nom, COUNT(1) "Nombre de films"
  FROM movies250 m
  JOIN movie_actor ma ON ma.movie_url  = m.url 
 WHERE ma.actor_name IN ("Morgan Freeman", "Tom Cruise")
 GROUP BY ma.actor_name WITH ROLLUP
 ORDER BY 2 DESC
"""

cur.execute(request)
results = cur.fetchall()

df = pd.DataFrame(results, columns=['Nom', 'Nombre de films'])

display(df)

,Nom,Nombre de films
0,TOTAL,101
1,Tom Cruise,61
2,Morgan Freeman,40


___
### 4) Quels sont les 3 meilleurs films d’horreur ? Dramatique ? Comique ?

In [76]:
genres = '"Comédie", "Drame", "Horreur"'

request = f"""
WITH ranked_movies AS (
                       SELECT m.title Titre, mg.genre_name Genre, m.score Note, 
                              ROW_NUMBER() OVER (PARTITION BY mg.genre_name ORDER BY m.score DESC) AS rank_pos
                         FROM movies250 m 
                         JOIN movie_genre mg ON mg.movie_url = m.url 
                        WHERE mg.genre_name IN ({genres}))
SELECT Titre, Genre, Note
  FROM ranked_movies
 WHERE rank_pos <= 3
"""

cur.execute(request)
results = cur.fetchall()

df = pd.DataFrame(results, columns=['Titre', 'Genre', 'Note'])

display(df)

,Titre,Genre,Note
0,The Counterfeit Kid,Comédie,9.7
1,The Home Game,Comédie,9.5
2,My Dead Friend Zoe,Comédie,9.5
3,My Dead Friend Zoe,Drame,9.5
4,Les évadés,Drame,9.3
5,Le Parrain,Drame,9.2
6,Wedding 93,Horreur,9.4
7,Desert Road,Horreur,9.4
8,Psychose,Horreur,8.5


___
### 5) Parmi les 100 films les mieux notés, quel pourcentage sont américains ? Français ?

In [77]:
countries = '"États-Unis", "France"'

request = f"""
SELECT IFNULL(m.country, "TOTAL") Pays, 
       COUNT(1) AS "Nombre de films",
       CONCAT(ROUND(COUNT(1) * 100.0 / (SELECT COUNT(1) FROM movies250), 2), "%") AS Pourcentage
  FROM movies250 m
WHERE m.country IN ({countries})
 GROUP BY m.country WITH ROLLUP
 ORDER BY 3 DESC
"""

cur.execute(request)
results = cur.fetchall()

df = pd.DataFrame(results, columns=['Pays', 'Nombre de films', 'Pourcentage'])

display(df)

,Pays,Nombre de films,Pourcentage
0,TOTAL,8293,68.34%
1,États-Unis,7970,65.68%
2,France,323,2.66%


___
### 6) Quel est la durée moyenne d’un film en fonction du genre ?

In [78]:
genres_excluded = '"Télé-réalité", "Jeu télévisé"'

request = f"""
SELECT mg.genre_name Genre,
       CONCAT(FLOOR(AVG(m.duration) / 60), 'h ', ROUND(MOD(AVG(m.duration), 60), 0), 'min') AS "Durée moyenne"
  FROM movie_genre mg
  JOIN movies250 m ON m.url = mg.movie_url
 WHERE mg.genre_name NOT IN ({genres_excluded})
 GROUP BY mg.genre_name 
 ORDER BY AVG(m.duration) DESC
"""

cur.execute(request)
results = cur.fetchall()

df = pd.DataFrame(results, columns=['Genre', 'Durée moyenne'])

display(df)

,Genre,Durée moyenne
0,Comédie Musicale,1h 49min
1,Science-fiction,1h 49min
2,Guerre,1h 47min
3,Action,1h 47min
4,Fantastique,1h 47min
5,Romantique,1h 46min
6,Biographique,1h 46min
7,Aventure,1h 46min
8,Historique,1h 45min
9,Drame,1h 43min


___
## Bonus :
### 7) En fonction du genre, afficher la liste des films les plus longs.

In [79]:
nombre_de_film = 1

request = f"""
WITH ranked_movies AS (
                       SELECT mg.genre_name Genre, m.title Titre, m.duration, 
                              ROW_NUMBER() OVER (PARTITION BY mg.genre_name ORDER BY m.duration DESC) AS rank_pos
                         FROM movie_genre mg 
                         JOIN movies250 m ON m.url  = mg.movie_url)
SELECT Genre, Titre, CONCAT(FLOOR(duration / 60), 'h ', ROUND(MOD(duration, 60), 0), 'min') "Durée"
  FROM ranked_movies
 WHERE rank_pos <= {nombre_de_film}
"""

cur.execute(request)
results = cur.fetchall()

phrase1 = f"Les {nombre_de_film} films"
phrase2 = "les plus longs"
if nombre_de_film == 1:
    phrase1 = "Le film"
    phrase2 = "le plus long"

genre = ''
for row in results:
    if genre != row[0]:
        genre = row[0]
        print()
        print(f"{phrase1} “{genre}” {phrase2} :")
    print("   -", row[1], ":", row[2])


Le film “Action” le plus long :
   - FPS: First Person Shooter : 4h 35min

Le film “Actualités” le plus long :
   - Ukraine Is Not a Brothel : 1h 18min

Le film “Animation” le plus long :
   - Batman: The Long Halloween : 2h 48min

Le film “Aventure” le plus long :
   - Flash Gordon : 4h 5min

Le film “Biographique” le plus long :
   - O.J.: Made in America : 7h 47min

Le film “Comédie” le plus long :
   - Un monde fou, fou, fou, fou : 3h 30min

Le film “Comédie Musicale” le plus long :
   - Lagaan : 3h 44min

Le film “Documentaire” le plus long :
   - Shoah : 9h 26min

Le film “Drame” le plus long :
   - Guerre et paix : 6h 33min

Le film “Famille” le plus long :
   - Flash Gordon : 4h 5min

Le film “Fantastique” le plus long :
   - Zack Snyder's Justice League : 4h 2min

Le film “Film noir” le plus long :
   - Pour que vivent les hommes : 2h 15min

Le film “Guerre” le plus long :
   - Shoah : 9h 26min

Le film “Historique” le plus long :
   - Shoah : 9h 26min

Le film “Horreur” le p

___
### 8) En fonction du genre, quel est le coût de tournage d’une minute de film ?

In [80]:
nombre_de_film = 1

request = f"""
WITH ranked_movies AS (
                       SELECT mg.genre_name Genre, m.title Titre, m.duration, 
                              ROW_NUMBER() OVER (PARTITION BY mg.genre_name ORDER BY m.duration DESC) AS rank_pos
                         FROM movie_genre mg 
                         JOIN movies250 m ON m.url  = mg.movie_url)
SELECT Genre, Titre, CONCAT(FLOOR(duration / 60), 'h ', ROUND(MOD(duration, 60), 0), 'min') "Durée"
  FROM ranked_movies
 WHERE rank_pos <= {nombre_de_film}
"""

cur.execute(request)
results = cur.fetchall()

phrase1 = f"Les {nombre_de_film} films"
phrase2 = "les plus longs"
if nombre_de_film == 1:
    phrase1 = "Le film"
    phrase2 = "le plus long"

genre = ''
for row in results:
    if genre != row[0]:
        genre = row[0]
        print()
        print(f"{phrase1} “{genre}” {phrase2} :")
    print("   -", row[1], ":", row[2])


Le film “Action” le plus long :
   - FPS: First Person Shooter : 4h 35min

Le film “Actualités” le plus long :
   - Ukraine Is Not a Brothel : 1h 18min

Le film “Animation” le plus long :
   - Batman: The Long Halloween : 2h 48min

Le film “Aventure” le plus long :
   - Flash Gordon : 4h 5min

Le film “Biographique” le plus long :
   - O.J.: Made in America : 7h 47min

Le film “Comédie” le plus long :
   - Un monde fou, fou, fou, fou : 3h 30min

Le film “Comédie Musicale” le plus long :
   - Lagaan : 3h 44min

Le film “Documentaire” le plus long :
   - Shoah : 9h 26min

Le film “Drame” le plus long :
   - Guerre et paix : 6h 33min

Le film “Famille” le plus long :
   - Flash Gordon : 4h 5min

Le film “Fantastique” le plus long :
   - Zack Snyder's Justice League : 4h 2min

Le film “Film noir” le plus long :
   - Pour que vivent les hommes : 2h 15min

Le film “Guerre” le plus long :
   - Shoah : 9h 26min

Le film “Historique” le plus long :
   - Shoah : 9h 26min

Le film “Horreur” le p

___
### 9) Quelles sont les séries les mieux notées ?

In [81]:
request = f"""
SELECT mg.genre_name Genre,
       CONCAT(FORMAT(AVG(m.budget / m.duration), 2), ' $/min') AS "Coût moyen/min",
       CONCAT(FORMAT(STDDEV(m.budget / m.duration), 2), ' $/min') AS "Écart type"
  FROM movie_genre mg
  JOIN movies250 m ON m.url = mg.movie_url
 WHERE m.duration > 0
   AND m.budget IS NOT NULL
 GROUP BY mg.genre_name
 ORDER BY 2 DESC
"""

cur.execute(request)
results = cur.fetchall()

df = pd.DataFrame(results, columns=['Genre', 'Coût moyen/min', 'Écart type'])

display(df)

,Genre,Coût moyen/min,Écart type
0,Aventure,"852,190.61 $/min","1,387,796.68 $/min"
1,Famille,"710,519.31 $/min","2,333,182.80 $/min"
2,Drame,"708,019.89 $/min","6,089,723.88 $/min"
3,Science-fiction,"696,120.47 $/min","561,195.73 $/min"
4,Fantastique,"648,370.22 $/min","749,602.09 $/min"
5,Documentaire,"61,953.31 $/min","239,041.17 $/min"
6,Horreur,"558,539.67 $/min","4,739,363.12 $/min"
7,Comédie,"557,529.51 $/min","530,263.50 $/min"
8,Romantique,"408,788.66 $/min","3,444,930.74 $/min"
9,Policier,"375,072.27 $/min","1,543,280.99 $/min"


### fermeture du curseur + connecteur

In [82]:
cur.close()
conn.close()